
<div hidden>

$\gdef\dd{\mathrm{d}}$

</div>

<div hidden>

$\gdef\abs#1{\left\vert#1\right\vert}$

</div>

<div hidden>

$\gdef\ve#1{\bm{#1}}$
</div>

<div hidden>

$\gdef\mat#1{\mathbf{#1}}$
</div>


# Two target distributions

In this notebook, we define two target distributions:
- a bi-variate "donut" distribution,
- and a posterior distribution for a 1D Poisson-based BIP.
that we will use in this chapter to illustrate sampling with CUQIpy. The former target is used for illustrative purposes and is not associated with an inverse problem, while the latter is a more realistic example of a BIP.

We also show high level usage of the `BayesianProblem` class to explore the posterior distribution as well as defining the target distribution in a more low-level approach through the `JointDistribution` class.









In [ ]:
from cuqi.distribution import DistributionGallery, Gaussian, JointDistribution
from cuqi.testproblem import Poisson1D
from cuqi.problem import BayesianProblem
import inspect
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH
import time
import scipy.stats as sps


In [ ]:

def plot2d(val, x1_min, x1_max, x2_min, x2_max, N2=201):
    # plot
    pixelwidth_x = (x1_max-x1_min)/(N2-1)
    pixelwidth_y = (x2_max-x2_min)/(N2-1)

    hp_x = 0.5*pixelwidth_x
    hp_y = 0.5*pixelwidth_y

    extent = (x1_min-hp_x, x1_max+hp_x, x2_min-hp_y, x2_max+hp_y)

    plt.imshow(val, origin='lower', extent=extent)
    plt.colorbar()


def plot_pdf_2D(distb, x1_min, x1_max, x2_min, x2_max, N2=201):
    N2 = 201
    ls1 = np.linspace(x1_min, x1_max, N2)
    ls2 = np.linspace(x2_min, x2_max, N2)
    grid1, grid2 = np.meshgrid(ls1, ls2)
    distb_pdf = np.zeros((N2,N2))
    for ii in range(N2):
        for jj in range(N2):
            distb_pdf[ii,jj] = np.exp(distb.logd(np.array([grid1[ii,jj], grid2[ii,jj]]))) 
    plot2d(distb_pdf, x1_min, x1_max, x2_min, x2_max, N2)

## <font color=#CD853F> The "donut" distribution </font> <a name="r-donut"></a>

In CUQIpy, we provide a set of bi-variate distributions for illustrative purposes. One of these is the "donut" distribution, which is a bi-variate distribution of a donut-shaped. The distribution is defined as follows:

$$

\begin{aligned}
log (p(\mathbf{x})) \propto - \frac{1}{\sigma_\text{donut}^2} \left( \left\| \mathbf{x} \right\| - r_\text{donut} \right)^2

\end{aligned}

$$

Where $\mathbf{x} = (x_1, x_2)$ is a 2D vector, $\left\| \mathbf{x} \right\|$ is the Euclidean norm of $\mathbf{x}$, $r_\text{donut}$ is the radius of the donut, and $\sigma_\text{donut}$ is a scalar value that controls the width of the "donut".

To load the "donut" distribution, we use the following:

In [ ]:

target_donut = DistributionGallery("donut")

print(target_donut)

We can plot the distribution probability density function (pdf):

In [ ]:
plot_pdf_2D(target_donut, -4, 4, -4, 4)


## <font color=#CD853F> A 1D Poisson-based BIP </font> <a name="r-donut"></a>

##### <font color=#8B4513> The forward model </font> <a name="r-forward-model"></a>

Consider a heat conductive rod of length $L = \pi$ with a varying conductivity (the conductivity of the rod changes from point to point). We fix the temperature at the end-points of the rod and apply a heat source distributed along the length of the rod. We wait until the rod reaches an equilibrium temperature distribution. The equilibrium temperature of the rod is modelled using the second order steady-state PDE as

$$
\left\{
\begin{aligned}
& \dfrac{\dd}{\dd \xi}\left(u(\xi) \dfrac{\dd y(\xi)}{\dd \xi}\right) = -f(\xi), \quad & \xi\in (0,L) \\
& y(0) = y(L) = 0.
\end{aligned}
\right.
$$
Here, $y$ represents the temperature distribution along the rod, $u(\xi) $ is the unknown conductivity of the rod and $f(\xi)$ is a deterministic heat source given by

$$
\begin{aligned}
	f(\xi) = 10\exp( -\frac{ (\xi - L/2)^2} {0.02} ).
\end{aligned}
$$

To ensure that the conductivity of the rod is non-negative, we parameterize $u$ by a random variable $x$ as follows:
 
$$
 \begin{aligned}
 u( \cdot  ) = \exp( x( \cdot  ) )
 \end{aligned}
$$
where $x$ is not necessarily positive.

Let us load the forward model that maps the random variable $x$ to the temperature distribution $y$ in CUQIpy. We will use the following parameters:
* `dim` : number of discretization points for the rod
* `L` : length of the rod
* `f` : a function that represents the heat source

In [ ]:
dim = 201
L = np.pi

The source term represents spikes at four locations `xs` with weight `ws`

In [ ]:
xs = np.array([0.2, 0.4, 0.6, 0.8])*L
ws = 0.8
sigma_s = 0.05
def f(t):
    s = np.zeros(dim-1)
    for i in range(4):
        s += ws * sps.norm.pdf(t, loc=xs[i], scale=sigma_s)
    return s

Let us plot the source term for visualization:

In [ ]:
temp_grid = np.linspace(0, L, dim-1)
plt.plot(temp_grid, f(temp_grid))

Then we can load the 1D Poisson forward model as follows:

In [ ]:
A, _, _ = Poisson1D(dim=dim, 
                    endpoint=L,
                    field_type='KL',
                    field_params={'num_modes': 10} ,
                    map=lambda x: np.exp(x), 
                    source=f).get_components()

We print the forward model to see its details.

In [ ]:
A

Let us look at the `pde` property of the forward model:

In [ ]:
A.pde

We can look at the domain and range geometries of the forward model.

In [ ]:
print(A.domain_geometry)
print(A.range_geometry)

And inspect the domain geometry further. Let us look at the mapping in the `MappedGeometry` object.

In [ ]:
print(inspect.getsource(A.domain_geometry.map))

We can extract the underlying geometry of the `MappedGeometry` object.

In [ ]:
underlying_geometry = A.domain_geometry.geometry
print(underlying_geometry)

The underlying geometry represents a Karhunen–Loève (KL) expansion of a random field. Let us look at some of the properties of this `underlying_geometry` such as the number of modes in the KL expansion and the grid on which the KL expansion basis functions are defined.

In [ ]:
print(A.domain_geometry.geometry.num_modes)
print(A.domain_geometry.geometry.grid)

The range geometry is of type `Continuous1D` which represents a 1D continuous signal/field defined on a grid. We can view the grid:

In [ ]:
print(A.domain_geometry.grid)

Additionally, the properties `domain_dim` and `range_dim` of the forward model represent the dimension of the input and output of the forward model, respectively.

In [ ]:
print(A.domain_dim)
print(A.range_dim)

##### <font color=#8B4513> The BIP: the prior </font> <a name="r-forward-model"></a>

We now build a posterior distribution based on this forward model. The unknown  $\mathbf{x}$ represents the coefficients in the KL expansion. We assume that the prior distribution of $\mathbf{x}$ is an i.i.d Gaussian distribution with mean $0$ and variance $\sigma_x^2$.

In [ ]:
sigma_x = 30
x = Gaussian(0, sigma_x**2, geometry=A.domain_geometry)


Let us assume that the true solution we want to infer is a sample from `x`. Note: we fix the random seed for reproducibility.

In [ ]:
np.random.seed(12)
x_true = x.sample()

##### <font color=#8B4513> Exercise </font> <a name="r-forward-model"></a>
- Visualize `x_true` in the KL coefficient space. Hint: try `x_true.plot(plot_par=True)`
- Visualize `x_true` in the corresponding function space (after applying the linear combination of KL basis weighted by KL vectors and then applying the exponantial mapping). Hint: all this can be achieved in one line by `x_true.plot(plot_par=False)`

In [ ]:
# your code here

##### <font color=#8B4513> The BIP: the likelihood </font> <a name="r-forward-model"></a>

We assume the data we obtain is a noisy measurement of the temperature $y$ over the interval $[0, L]$ in all grid points. The measurements form a vector $\mathbf{y}$. The noise is assumed to be additive Gaussian noise with mean $0$ and variance $\sigma_y^2$.

$$
\mathbf{y} = \mathbf{A}(\mathbf{x}) + \epsilon  \quad \text{where} \quad \epsilon \sim \mathcal{N}(0, \sigma_y^2).

$$

We define the data distribution $p(\mathbf{y} | \mathbf{x})$ in `CUQIpy` in this case as

In [ ]:
sigma_y = np.sqrt(0.001)
y = Gaussian(A(x), sigma_y**2, geometry=A.range_geometry)

We create a synthetic data to use it to test solving our BIP.  We denote this data as $\mathbf{y}_{\text{obs}}$ which is a particular observed data realization from a setup where the KL coefficients are `x_true`. To create this data in `CUQIpy`, we use the following:

In [ ]:
y_obs = y(x=x_true).sample()

Let us plot the true conductivity field, corresponding to `x_true`, the data `y_true` without noise i.e. `A(x_true)`, and the noisy data `y_obs` in the same plot.

In [ ]:
y_obs.plot(label='y_obs')
A(x_true).plot(label='y_true')
x_true.plot(label='x_true')
plt.legend()

##### <font color=#8B4513> The BIP: the posterior distribution  (the high level approach: using the BayesianProblem class)</font> 

The posterior distribution of the Bayesian inverse problem in this case is given by

$$
\begin{align*}
p(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{obs}) \propto L(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{obs})p(\mathbf{x}),
\end{align*}
$$

where we use the notation $L(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{obs}) := p(\mathbf{y}=\mathbf{y}_\mathrm{obs} \mid \mathbf{x})$ for the likelihood function to emphasize that, in the context of the posterior where $\mathbf{y}$ is fixed to $\mathbf{y}_\mathrm{obs}$, it is a function of $\mathbf{x}$ and not on $\mathbf{y}$. In CUQIpy we sometimes use the short-hand printing style `L(x|y)` for brevity.



The simplest way to sample a Bayesian inverse problem in CUQIpy is to use the [BayesianProblem class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.html#cuqi.problem.BayesianProblem).

Using the BayesianProblem class, one can easily define and sample from the posterior distribution of a Bayesian inverse problem by providing the distributions for the parameters and data and subsequently setting the observed data.

In [ ]:
BP_poisson = BayesianProblem(x, y)      # Create Bayesian problem
BP_poisson.set_data(y=y_obs)           # Provide observed data

In the above example, we provided our assumptions about the data generating process by defining the distributions for the parameters and data and provided the observed data for the problem. `CUQIpy` internally created the posterior distribution using the provided distributions and data. 

We can use this object to sample from the posterior distribution using the `UQ` method, which we will experiment with in this exercise:



##### <font color=#8B4513> Exercise </font> <a name="r-forward-model"></a>
Use the `UQ` method of the `BP_poisson` object to sample the posterior distribution. The `UQ` returns a `Samples` object, store the result in a variable called `BP_poisson_samples`.

In [ ]:
# your code here

In the previous exercise we saw that `CUQIpy` automatically decided on using a sampler, preconditioned Crank Nicolson `pCN` in this case, and sampled the posterior distribution. Additionally, the  credibility interval for the parameter $\mathbf{x}$ as well as the mean of the posterior was plotted and compared to the ground truth (`x_true`).

**Note about visualizing the credible interval**:
Using the `UQ` method, the credibility interval is computed for the KL coefficients. Then mapped to the function space and plotted. We can also compute the credibility interval directly on the function values. We will revisit this at a later stage.


In the next section, we show how to define the posterior distribution more explicitly. 

##### <font color=#8B4513> The BIP: computing the MAP point with the BayesianProblem class</font>


In addition to sampling the posterior, we can also compute point estimates of the posterior. A common point estimate to consider is the Maximum A Posteriori (MAP) estimate, which is the value of the Bayesian parameter that maximizes the posterior density. That is,

$$
\begin{align*}
\mathbf{x}_\mathrm{MAP} = \arg\max_\mathbf{x} p(\mathbf{x} \mid \mathbf{y}_\mathrm{obs}).
\end{align*}
$$

The easiest way to compute the MAP estimate in CUQIpy is to use the `MAP` method of the `BayesianProblem` class as follows:

In [ ]:
# x_map = BP_poisson.MAP()

We can plot the MAP estimate and compare it to the true solution `x_true`.

In [ ]:
# x_map.plot(label='MAP estimate')
x_true.plot(label='True solution')
plt.legend()

We can also look at the MAP estimate in the KL coefficient space:

In [ ]:
# x_map.plot(label='MAP estimate' ,plot_par=True)
x_true.plot(label='True solution' ,plot_par=True)
plt.legend()

We notice that in general the larger the mode number, the harder it is to be inferred (can you think why?).

##### <font color=#8B4513> The BIP: the posterior distribution  (the low level approach: using the JointDistribution)</font> 


To define the posterior distribution explicitly in CUQIpy, we first define the joint distribution $p(\mathbf{y},\mathbf{x})$, then we supply the observed data to create the conditional distribution $p(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{obs})$.


Let us first define the joint distribution $p(\mathbf{y},\mathbf{x})$ in CUQIpy. We use the following:

In [ ]:
# Define joint distribution p(y,x)
joint = JointDistribution(y, x)

Calling `print` on the joint distribution gives a nice overview matching the mathematical description of the joint distribution.

In [ ]:
print(joint)

CUQIpy can automatically derive the posterior distribution for any joint distribution when we pass the observed data as an argument to the "call" (condition) method of the joint distribution.  This is done as follows:

In [ ]:
target_poisson = joint(y=y_obs) # Condition p(x,y) on y=y_data. Applies Bayes' rule automatically


We can now inspect the posterior distribution by calling `print` on it. Notice that the posterior equation matches the mathematical expression we showed above.

In [ ]:
print(target_poisson)


##### <font color=#8B4513> Exercise </font> <a name="r-forward-model"></a>
- The posterior is essentially just another CUQIpy distribution. Have a look at the [Posterior class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.Posterior.html) in the online documentation to see what attributes and methods are available.

- Try evaluating the posterior log probability density function (logpdf) and pdf at some points say `x_true` and `x_true*1.1`.


In [ ]:
# Your code here